In [57]:
import pandas as pd

# date and time handling
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

In [58]:
sites = pd.read_csv('non_deleted 11_6_2018.csv')
ga = pd.read_csv('Analytics All Web Site Data Content Drilldown 20171105-20181105.csv', thousands=',')
ts = pd.read_csv('theme status.csv')
st = pd.read_csv('updateSiteType 20180917.csv')
ps = pd.read_csv('plugin status.csv')
pl = pd.read_csv('plugins 11_6_2018.csv')

In [59]:
# theme accessibility status
sites = pd.merge(sites, ts, on='current_theme')

In [61]:
# clean and merge GA data
ga.drop(['Bounce Rate', '% Exit'], axis=1, inplace=True)
sites = pd.merge(sites, ga, left_on='blog_url', right_on='Page path level 1', how='left')
sites.drop(['Page path level 1'], axis=1, inplace=True)

# set NaNs to 0 - no traffic on site
sites[['Pageviews', 'Unique Pageviews', 'Avg. Time on Page']] \
= sites[['Pageviews', 'Unique Pageviews', 'Avg. Time on Page']].fillna(0)

In [63]:
# bring in manually categorized sites
sites.drop('site_type', axis=1,inplace=True)
st.drop('url', axis=1, inplace=True)
sites = pd.merge(sites, st, on='blog_id')

In [64]:
# bring in plugins with activation status and accessibility counts for each site
'''
plugin_list is derived from:
int64_columns = pl.select_dtypes([np.number]).columns
int64_columns
'''
plugin_list = ['aesop-story-engine', 'akismet', 'attachment-importer',
       'autoblog', 'bbpress', 'bwp-minify', 'comment-image',
       'contus-video-gallery', 'custom-sidebars', 'display-posts-shortcode',
       'duplicate-post', 'editorial-access-manager', 'email-users',
       'embed-webmap', 'enable-media-replace', 'every-calendar-1',
       'feedwordpress', 'footnotes', 'google-document-embedder', 'gutenberg',
       'iframe', 'image-wall', 'import-users-from-csv', 'jetpack',
       'leaflet-maps-marker', 'legacy-jetpack-custom-css-editor', 'lock-posts',
       'login-logout-register-menu', 'nextgen-gallery',
       'photo-video-gallery-master', 'pinterest-rss-widget',
       'post-type-switcher', 'shibboleth-fix', 'simple-mathjax',
       'subscribe-by-email', 'syntaxhighlighter', 'the-events-calendar',
       'tt-guest-post-submit', 'tumblr-widget-for-wordpress',
       'user-submitted-posts', 'wp-accessibility', 'wp-gallery-custom-links',
       'wp-pro-quiz', 'wp-recaptcha', 'wp-views', 'wpmu-dev-post-votes',
       'wpmu_dev_maps_plugin', 'yop-poll', 'zotpress',
        'scriptless-social-sharing','social-icons']

# iterate through columns and convert data type to bool
for col in plugin_list:
    pl[col] = pl[col].astype(bool)
    
# create list of inaccessible, user-activated plugins
f_inac_p = ps['Plugin'].loc[(ps['Frontend Accessible'] == 'No') & \
                                (ps['Network Active'] == 'No')].tolist()

# create list of all inaccessible plugins
inac_p = ps['Plugin'].loc[ps['Accessible'] == 'No'].tolist()

# count active plugins on site
pl['total active'] = pl[plugin_list].sum(axis=1)

# count inaccessible, user activated plugins per site
pl['front inacc plugins'] = pl[f_inac_p].sum(axis=1)
pl['inacc plugins'] = pl[inac_p].sum(axis=1)

# prep plugins df to merge into main sites df
pl.drop(['domain', 'path', 'registered', 'last_updated', 'admin_email', 'theme'], axis=1, inplace=True)

# join to sites df
sites = pd.merge(sites, pl, on='blog_id')

In [91]:
sites.columns.values

array(['blog_id', 'blog_name', 'blog_description', 'siteurl', 'blog_url',
       'privacy descr', 'has_accessibility_footer', 'admin_email',
       'users_count', 'active_plugins_count', 'db_version',
       'current_theme', 'custom_css', 'registered', 'last_updated',
       'archived', 'posts_count', 'pages_count', 'comments_count',
       'attachments_count', 'owner', 'site owner role', 'admin users',
       'template', 'Google Drive embeds (count)',
       'Google Docs embeds (count)', 'Google Sheets embeds (count)',
       'Google Slides embeds (count)', 'Google Calendar embeds (count)',
       'Google Groups embeds (count)', 'Google Hangouts embeds (count)',
       'Google Sites embeds (count)', 'NYU Stream embeds (count)',
       'accessible', 'Pageviews', 'Unique Pageviews', 'Avg. Time on Page',
       'site type', 'aesop-story-engine', 'akismet',
       'attachment-importer', 'auto-subscribe-users', 'autoblog',
       'avatars', 'bbpress', 'blogtemplates', 'bwp-minify',
       

In [65]:
# convert registered and lastmod to datetime for analysis
sites['registered'] = pd.to_datetime(sites['registered'])
sites['last_updated'] = pd.to_datetime(sites['last_updated'])

In [66]:
'''
create function to define active sites
r = creation (registered) timestamp
l = last_updated timestamp
ap = admin pageviews
tp = total pageviews

r_date = will check if site was created more than 1yr before this date
l_date = will check if site was modified more than 1yr before this date
tp_min = minimum number of total pageviews to be considered active
ap_min = minimum number of admin pageviews to be considered active
''' 

r_date = date(2018,11,5) 
l_date = date(2018,11,5)
tp_min = 2

def check_active(r, l, tp):
    if r < pd.Timestamp(r_date) - relativedelta(years=1) and \
    l < pd.Timestamp(l_date) - relativedelta(years=1) and \
    tp < tp_min:
        return False 
    else:
        return True

In [67]:
# apply check_active to all sites and mark with bool
sites['active'] = sites.apply(lambda row: check_active(row['registered'], row['last_updated'], row['Pageviews']),axis=1)


In [86]:
# convert 'has_accessibility_footer' to bool.
# all archived sites and other N/As will become False
sites['has_accessibility_footer'] = sites['has_accessibility_footer']=='true'

In [95]:
# Question: how many active public facing inaccessible sites do we have for each site type?
sites['site type'].loc[(sites['active'] == True) & \
                      (sites['accessible'] == False) &\
                      (sites['has_accessibility_footer'] == False) &\
                      (sites['inacc plugins'] > 0) &\
                      ((sites['privacy descr'] == 'Public') |\
                       (sites['privacy descr'] == 'Hidden'))].value_counts()

Teaching & Learning    306
Portfolio              268
Administrative         107
Research                79
Site Deleted             1
Name: site type, dtype: int64

In [97]:
# Question: how many total public facing inaccessible sites do we have for each site type?
sites['site type'].loc[((sites['accessible'] == False) |\
                      (sites['has_accessibility_footer'] == False) |\
                      (sites['inacc plugins'] > 0)) &\
                      ((sites['privacy descr'] == 'Public') |\
                       (sites['privacy descr'] == 'Hidden'))].value_counts()

Teaching & Learning    1470
Portfolio              1295
Administrative          319
Research                302
Site Deleted             66
Name: site type, dtype: int64

In [100]:
# Question: how many public (and search indexed) inaccessible sites do we have for each site type?
sites['site type'].loc[((sites['accessible'] == False) |\
                      (sites['has_accessibility_footer'] == False) |\
                      (sites['inacc plugins'] > 0)) &\
                      (sites['privacy descr'] == 'Public')].value_counts()

Portfolio              963
Teaching & Learning    772
Research               245
Administrative         193
Site Deleted            48
Name: site type, dtype: int64

In [103]:
# Question: how many public inaccessible sites do we have for each site type?
# without plugins
sites['site type'].loc[((sites['accessible'] == False) |\
                      (sites['has_accessibility_footer'] == False) |\
                      (sites['inacc plugins'] == 0)) &\
                      (sites['privacy descr'] == 'Public')].value_counts()

Portfolio              949
Teaching & Learning    747
Research               238
Administrative         176
Site Deleted            48
Name: site type, dtype: int64

In [105]:
# Question: how many inactive public inaccessible sites do we have for each site type?
# without plugins
sites['site type'].loc[(sites['active'] == False) & \
                       ((sites['accessible'] == False) |\
                      (sites['has_accessibility_footer'] == False) |\
                      (sites['inacc plugins'] > 0)) &\
                      (sites['privacy descr'] == 'Public')].value_counts()

Portfolio              318
Teaching & Learning    215
Research                58
Administrative          25
Name: site type, dtype: int64